<a href="https://colab.research.google.com/github/swarnitav08/BaggageAI_CV_Assignment/blob/main/BaggageAI_CV_Hiring_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##BaggageAI Computer Vision Hiring Assignment
Case Study:- Image Processing

You are given two sets of images:- background and threat objects. Background images are the background x-ray images of baggage that gets generated after passing through a X-ray machine at airport. Threat images are the x-ray images of threats that are prohibited at airport while travelling.

• Your task is to cut the threat objects, scale it down, rotate with 45 degree and paste it into the background images using image processing techniques in python.

• Threat objects should be translucent, means it should not look like that it is cut pasted. It should look like that the threat was already there in the background images. Translucent means the threat objects should have shades of background where it is pasted.

• Threat should not go outside the boundary of the baggage. **difficult**

• If there is any background of threat objects, then it should not be cut pasted into the background images, which means while cutting the threat objects, the boundary of a threat object should be tight-bound.

##INTRODUCTION

Various Image processing techniques have been used to complete the above given task. The libraries used are OpenCV and PIL (Python Imaging Library) along with imutils and glob. The order followed for taking threat images is as per given in the folders provided. 

1) Transparency is maintained at 65% for all threat images so that they appear translucent after pasting. Concept of masking has been used to isolate threat object from its background.

2) Scaling down (resizing) has been done based on coresponding background image height has been done for all threat images.

3) Rotate with 45 degree has been done for all threat images.

Author: Swarnita Venkatraman

Date: 02/11/2020

In [2]:
#uploading entire zip file at once
from google.colab import files
uploaded = files.upload()

Saving BaggageAI_CV_Hiring_Assignment.zip to BaggageAI_CV_Hiring_Assignment.zip


In [3]:
#unzipping the file
!unzip BaggageAI_CV_Hiring_Assignment.zip

Archive:  BaggageAI_CV_Hiring_Assignment.zip
   creating: BaggageAI_CV_Hiring_Assignment/
  inflating: BaggageAI_CV_Hiring_Assignment/BaggageAI_CV_Hiring_assignment.pdf  
   creating: BaggageAI_CV_Hiring_Assignment/background_images/
  inflating: BaggageAI_CV_Hiring_Assignment/background_images/BAGGAGE_20180811_175323_83216_B_1.jpg  
  inflating: BaggageAI_CV_Hiring_Assignment/background_images/BAGGAGE_20180811_175328_83216_A_1.jpg  
  inflating: BaggageAI_CV_Hiring_Assignment/background_images/S0210209058_20180811232942_L-1_1.jpg  
  inflating: BaggageAI_CV_Hiring_Assignment/background_images/S0300542812_20180822020845_L-10_1.jpg  
  inflating: BaggageAI_CV_Hiring_Assignment/background_images/S0320365070_20180821160850_L-12_5.jpg  
   creating: BaggageAI_CV_Hiring_Assignment/threat_images/
  inflating: BaggageAI_CV_Hiring_Assignment/threat_images/BAGGAGE_20170522_113049_80428_A.jpg  
  inflating: BaggageAI_CV_Hiring_Assignment/threat_images/BAGGAGE_20170522_115645_80428_B.jpg  
  infl

In [4]:
#importing necessary libraries
import cv2
import imutils
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
import numpy

In [5]:
#storing all threat images in image_list
image_list = []
for filename in glob.glob("BaggageAI_CV_Hiring_Assignment/threat_images/*.jpg"):
    im=Image.open(filename)
    image_list.append(im)

In [6]:
image_list

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=147x832 at 0x7F7B8F456C88>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=157x768 at 0x7F7B8F3E6080>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=347x768 at 0x7F7B8F3E6EB8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=273x768 at 0x7F7B8F44C320>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=232x768 at 0x7F7B8F44C390>]

Making mask transparent so that threat image appears translucent

In [7]:
masked=[]
image_no=1
TRANSPARENCY=65 #setting transparency percentage
for img in image_list:
  mask_2=img.convert("L")
  th=248 # the value adjusted based on all images
  mask_2 = mask_2.point(lambda i: i < th and 255)
  paste_mask_2 = mask_2.split()[0].point(lambda i: i * TRANSPARENCY / 100.) #splitting into individual bands and multiplying each pixel by transparency
  masked.append(paste_mask_2)
  name = 'paste_mask' + str(image_no) + '.png'
  paste_mask_2.save(name, 'PNG')
  image_no += 1

In [8]:
masked #saving all paste masks in a list 'masked'

[<PIL.Image.Image image mode=L size=147x832 at 0x7F7B8F4085C0>,
 <PIL.Image.Image image mode=L size=157x768 at 0x7F7B8F408630>,
 <PIL.Image.Image image mode=L size=347x768 at 0x7F7B8F4085F8>,
 <PIL.Image.Image image mode=L size=273x768 at 0x7F7B8F408668>,
 <PIL.Image.Image image mode=L size=232x768 at 0x7F7B8F4086A0>]

In [9]:
#saving height of background image in a list 'images_list' to resize corresponding threat image
images_list=[]
for filename in glob.glob("BaggageAI_CV_Hiring_Assignment/background_images/*.jpg"):
    im=Image.open(filename)
    images_list.append(im.size[1])

In [10]:
images_list

[630, 191, 793, 600, 377]

Resizing and Rotating threat image

In [11]:
image_no=1 #initialising

for (filenamee,i) in zip(glob.glob("BaggageAI_CV_Hiring_Assignment/threat_images/*.jpg"),images_list):
    images=cv2.imread(filenamee)  
    imagee = imutils.resize(images,height=i) #set corresponding height
    rotate = imutils.rotate_bound(imagee, 45) #rotate by 45 degrees
    name = 'resized' + str(image_no) + '.png'
    cv2.imwrite(name, rotate)
    image_no += 1

Resizing and Rotating mask

In [12]:
image_no=1

for (filenamee,i) in zip(masked,images_list):
    images = cv2.cvtColor(numpy.array(filenamee),cv2.COLOR_RGB2BGR) #reading using cv2 after converting to numpy array in colour as images saved in PIL format inside 'masked'
    imagee = imutils.resize(images,height=i) #If you need to modify the image resolution and keep your aspect ratio use the function imutils
    rotate = imutils.rotate_bound(imagee, 45)
    name = 'resized_mask' + str(image_no) + '.png'
    cv2.imwrite(name, rotate)
    image_no += 1

Pasting threat image on background image

In [14]:
l=[1,2,3,4,5] #for iterating

In [15]:
image_no=1

for (filenamee,i,j) in zip(glob.glob("BaggageAI_CV_Hiring_Assignment/background_images/*.jpg"),l,l):
  im1=Image.open(filenamee) #background image
  back_im = im1.copy()
  im2 = Image.open('resized'+str(i)+'.png') #resized and rotated threat image
  mask_im = Image.open('resized_mask'+str(j)+'.png') #resized and rotated mask
  mask_imm=mask_im.convert('L')#making it single channel image
  #When the mask image is 8-bit gray scale (mode='L'), the base image is output when the value of the mask image is 0 (black), 
  #and the pasted image is output when 255 (white).
  back_im.paste(im2, (48,40), mask_imm) #pasting 
  name = 'final' + str(image_no) + '.png'
  back_im.save(name, quality=95)
  image_no += 1